In [2]:
import numpy as np
import pandas as pd

# Suppress future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
vehicles_list = ['ICEV', 'PHEV', 'BEV', 'FCEV']
demand_list = ['normal','EU100','EU0']

In [4]:
# SECOND PHASE parsing -- assume outlook stock is ready from well to tank parsing
# translate the outlook into output requirements
file_path = "data/outlook_vehicles_stock.csv"
outlook = pd.read_csv(file_path)
outlook = pd.melt(outlook, id_vars=['category','region','year'], value_vars=['ICEV', 'PHEV', 'BEV', 'FCEV'],
                    var_name='vehicles', value_name='outlook')

In [5]:
#assuming that battery life cycle = 5 years
battery_age = 5
outlook["gap_year"] = outlook.groupby(['category','vehicles'])["year"].diff()
outlook["gap_stock"] = outlook.groupby(['category','vehicles'])["outlook"].diff()
outlook["avg_growth"] = outlook["gap_stock"]/outlook["gap_year"]

outlook["battery_outlook"] = outlook["outlook"]-(outlook["avg_growth"]*battery_age) 
outlook.loc[outlook['battery_outlook'] < 0, 'battery_outlook'] = 0

outlook = outlook.drop(columns=["region","gap_year","gap_stock","avg_growth"])

In [6]:
outlook.to_csv("data/outlook_battery_stock.csv",index=False)

In [7]:
file_path = "inputs/input_battery.csv"
requirement = pd.read_csv(file_path)
requirement = pd.melt(requirement, id_vars=['Product'], value_vars=['ICEV', 'PHEV', 'BEV', 'FCEV'],
                    var_name='vehicles', value_name='requirement')
requirement.rename(columns={'Product': 'sector'},inplace=True)

In [8]:
file_path = "inputs/input_market_shares.csv"
market_shares = pd.read_csv(file_path)
market_shares = pd.melt(market_shares, id_vars=["scenario","region"],var_name="vehicles",value_vars=vehicles_list, value_name="shares")
market_shares.rename(columns={"scenario":"demand"},inplace=True)

In [9]:
outlook_requirement = pd.merge(outlook, requirement, on='vehicles', how='outer')
outlook_requirement_shares = pd.merge(market_shares, outlook_requirement, on='vehicles', how='outer')
outlook_requirement_shares["value"] = outlook_requirement_shares["battery_outlook"] * outlook_requirement_shares["requirement"] * outlook_requirement_shares["shares"]

In [10]:
outlook_sectors = outlook_requirement_shares.pivot_table(index=['category','demand','region', 'year', 'sector'],
                          columns='vehicles',
                          values='value',
                          aggfunc='sum')

In [11]:
outlook_sectors.groupby(["category","demand","region"]).sum("BEV")

vehicles                                BEV        FCEV  ICEV         PHEV
category         demand region                                            
Projection-APS   EU0    EU           0.0000    0.000000   0.0      0.00000
                        Non EU  392121.6000  255.780000   0.0  41834.01000
                 EU100  EU      392121.6000  255.780000   0.0  41834.01000
                        Non EU       0.0000    0.000000   0.0      0.00000
                 normal EU       83521.9008   54.481140   0.0   8910.64413
                        Non EU  308599.6992  201.298860   0.0  32923.36587
Projection-STEPS EU0    EU           0.0000    0.000000   0.0      0.00000
                        Non EU  361958.4000  156.290400   0.0  45280.29000
                 EU100  EU      361958.4000  156.290400   0.0  45280.29000
                        Non EU       0.0000    0.000000   0.0      0.00000
                 normal EU       77097.1392   33.289855   0.0   9644.70177
                        Non EU  284861.2608  123.000545   0.0  35635.58823

In [12]:
scenario_list = ["Projection-APS","Projection-STEPS"]
year_list = [2023,2025,2030,2035]
#scenario = scenario_list[0]
#demand = demand_list[0]

for scenario in scenario_list:
    for demand in demand_list: 
        outlook_scenario = outlook_sectors.reset_index()
        outlook_scenario = outlook_scenario[(outlook_scenario.category==scenario)&(outlook_scenario.demand==demand)]
        outlook_scenario = outlook_scenario[["region","year","sector","ICEV","PHEV","BEV","FCEV"]]
        
        file_name = "data/outlook_battery_"+scenario+"_"+demand+".csv"
        outlook_scenario.to_csv(file_name,index=False)